In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
filenames=['/Users/krhee/Dropbox (ASU)/Coding/RA_Coding/providerservice/MUP_PHY_R19_P04_V10_D13_Prov_Svc.csv',
          '/Users/krhee/Dropbox (ASU)/Coding/RA_Coding/providerservice/MUP_PHY_R19_P04_V10_D14_Prov_Svc.csv',
          '/Users/krhee/Dropbox (ASU)/Coding/RA_Coding/providerservice/MUP_PHY_R19_P04_V10_D15_Prov_Svc.csv',
          '/Users/krhee/Dropbox (ASU)/Coding/RA_Coding/providerservice/MUP_PHY_R19_P04_V10_D16_Prov_Svc.csv',
          '/Users/krhee/Dropbox (ASU)/Coding/RA_Coding/providerservice/MUP_PHY_R19_P04_V10_D17_Prov_Svc.csv',
           '/Users/krhee/Dropbox (ASU)/Coding/RA_Coding/providerservice/MUP_PHY_R20_P04_V10_D18_Prov_Svc.csv',
           '/Users/krhee/Dropbox (ASU)/Coding/RA_Coding/providerservice/MUP_PHY_R21_P04_V10_D19_Prov_Svc.csv']

Here are all the HCPCS Codes for years 2013 ~ 2019

In [26]:
# new_df = pd.DataFrame()
# for i in range(len(filenames)):
#     temp_df = pd.read_csv(filenames[i], encoding='latin1')
#     temp_df = temp_df[['HCPCS_Cd','HCPCS_Desc']].drop_duplicates().reset_index(drop=True)
#     temp_df['year']= int(str('20')+filenames[i].split("_D")[-1][:2])
#     new_df = new_df.append(temp_df).reset_index(drop=True)

/Users/krhee/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [28]:
# new_df.to_csv(r'del.csv',index=False)

##### For ambulance

In [3]:
new_df = pd.DataFrame()
for i in range(len(filenames)):
    temp_df = pd.read_csv(filenames[i], encoding='latin1')
    temp_df = temp_df[temp_df['HCPCS_Cd'].isin(['A0425','A0426','A0427','A0428','A0429','A0430','A0431',
                                               'A0432','A0433','A0434','A0435','A0436','A0999'])].reset_index(drop=True)
    temp_df['year']= int(str('20')+filenames[i].split("_D")[-1][:2])
    new_df = new_df.append(temp_df).reset_index(drop=True)

/Users/krhee/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
new_df = new_df[['Rndrng_NPI','year','Rndrng_Prvdr_Last_Org_Name','HCPCS_Cd',
                         'Rndrng_Prvdr_State_Abrvtn','Rndrng_Prvdr_State_FIPS', 'Rndrng_Prvdr_Zip5',
                         'Tot_Benes','Tot_Srvcs','Tot_Bene_Day_Srvcs','Avg_Sbmtd_Chrg']]
new_df['Tot_Srvcs_Chrg'] = new_df['Tot_Srvcs']*new_df['Avg_Sbmtd_Chrg']
new_df = new_df.drop(columns=['Avg_Sbmtd_Chrg'])
new_df.columns=['npi','year','lastname','hcpcs','state_id','state_fips','zip','tot_benes','tot_srvcs','tot_bene_day_srvcs','tot_srvcs_chrg']

/var/folders/ff/7xvchf4558v3w_jjs11bdwlw0000gp/T/ipykernel_25227/546155700.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Tot_Srvcs_Chrg'] = new_df['Tot_Srvcs']*new_df['Avg_Sbmtd_Chrg']


In [10]:
new_df_copy = new_df.copy()

In [7]:
# First, check if zip is missing --> All XXs
# new_df[new_df['zip'].isna()]
new_df = new_df[~new_df['zip'].isna()].reset_index(drop=True)
# Then let's match zip code to state
new_df['zip_3'] = [str(i)[:3] for i in new_df['zip']]
# Drop weird
new_df = new_df[~new_df['zip_3'].isin(['0','0.0', 'EH1','LA1','LA2','N9G'])].reset_index(drop=True)
new_df = new_df[~new_df['state_fips'].isna()].reset_index(drop=True)

In [8]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231856 entries, 0 to 231855
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   npi                 231856 non-null  int64  
 1   year                231856 non-null  int64  
 2   lastname            231856 non-null  object 
 3   hcpcs               231856 non-null  object 
 4   state_id            231856 non-null  object 
 5   state_fips          231856 non-null  float64
 6   zip                 231856 non-null  object 
 7   tot_benes           231856 non-null  int64  
 8   tot_srvcs           231856 non-null  float64
 9   tot_bene_day_srvcs  231856 non-null  int64  
 10  tot_srvcs_chrg      231856 non-null  float64
 11  zip_3               231856 non-null  object 
dtypes: float64(3), int64(4), object(5)
memory usage: 21.2+ MB


In [11]:
# Check
zip_df = new_df[['zip','state_id','state_fips']].drop_duplicates()
zip_df.groupby(['zip']).nunique().sort_values(by=['state_id'])
zip_df.groupby(['zip']).nunique().sort_values(by=['state_fips'])
# State FIPS is more accurate --> Drop State ID
new_df = new_df.drop(columns=['state_id'])

In [13]:
# New state ID
state_dict = {1:'AL',2:'AK',4:'AZ',5:'AR',6:'CA',8:'CO',9:'CT',10:'DE',11:'DC',
              12:'FL',13:'GA',15:'HI',16:'ID',17:'IL',18:'IN',19:'IA',20:'KS',
              21:'KY',22:'LA',23:'ME',24:'MD',25:'MA',26:'MI',27:'MN',28:'MS',
              29:'MO',30:'MT',31:'NE',32:'NV',33:'NH',34:'NJ',35:'NM',36:'NY',
              37:'NC',38:'ND',39:'OH',40:'OK',41:'OR',42:'PA',44:'RI',45:'SC',
              46:'SD',47:'TN',48:'TX',49:'UT',50:'VT',51:'VA',53:'WA',54:'WV',
              55:'WI',56:'WY'}
state_df = pd.DataFrame.from_dict(data=state_dict, orient='index').reset_index(level=[0])
state_df.columns=['state_fips','state_id']
new_df = pd.merge(new_df, state_df, on=['state_fips'])

In [14]:
# Data types (important)
new_df['npi'] = new_df['npi'].astype(int)
new_df['year'] = new_df['year'].astype(int)
new_df['zip'] = new_df['zip'].astype(float)
new_df['zip'] = new_df['zip'].astype(int)
new_df['state_fips'] = new_df['state_fips'].astype(int)

In [15]:
# 1. Non-emergency ambulance (A0426, A0428, A0434)

new_df_1 = new_df[new_df['hcpcs'].isin(['A0426', 'A0428', 'A0434'])][['npi','year','lastname','state_id','state_fips','zip','tot_benes','tot_srvcs','tot_bene_day_srvcs','tot_srvcs_chrg']].drop_duplicates().reset_index(drop=True)
new_df_1_gb = new_df_1.groupby(['npi','lastname','state_id','state_fips','zip','year']).sum().reset_index(level=[0,1,2,3,4,5])
new_df_1_gb_gb = new_df_1_gb.groupby(['zip','year','state_id','state_fips']).sum()[['tot_benes','tot_srvcs','tot_bene_day_srvcs','tot_srvcs_chrg']].reset_index(level=[0,1,2,3])
new_df_1 = new_df_1_gb_gb.copy()

In [16]:
new_df_1.groupby(['zip','year','state_id','state_fips']).size().sort_values()

zip    year  state_id  state_fips
1057   2014  MA        25            1
63068  2019  MO        29            1
       2018  MO        29            1
       2017  MO        29            1
       2016  MO        29            1
                                    ..
32448  2013  FL        12            1
32435  2018  FL        12            1
       2017  FL        12            1
32448  2018  FL        12            1
99925  2019  AK        2             1
Length: 19770, dtype: int64

In [17]:
new_df_1.to_csv(r'nonemerg_amb.csv', index=False)

In [18]:
# 2. Emergency ambulance (A0427, A0429)

new_df_2 = new_df[new_df['hcpcs'].isin(['A0427', 'A0429'])][['npi','year','lastname','state_id','state_fips','zip','tot_benes','tot_srvcs','tot_bene_day_srvcs','tot_srvcs_chrg']].drop_duplicates().reset_index(drop=True)
new_df_2_gb = new_df_2.groupby(['npi','lastname','state_id','state_fips','zip','year']).sum().reset_index(level=[0,1,2,3,4,5])
new_df_2_gb_gb = new_df_2_gb.groupby(['zip','year','state_id','state_fips']).sum()[['tot_benes','tot_srvcs','tot_bene_day_srvcs','tot_srvcs_chrg']].reset_index(level=[0,1,2,3])
new_df_2 = new_df_2_gb_gb.copy()

In [19]:
new_df_2.groupby(['zip','year','state_id','state_fips']).size().sort_values()

zip    year  state_id  state_fips
1001   2013  MA        25            1
60018  2015  IL        17            1
       2016  IL        17            1
       2017  IL        17            1
       2018  IL        17            1
                                    ..
30269  2017  GA        13            1
       2018  GA        13            1
       2019  GA        13            1
30214  2014  GA        13            1
99929  2019  AK        2             1
Length: 53501, dtype: int64

In [20]:
new_df_2.to_csv(r'emerg_amb.csv', index=False)

In [21]:
# 3. All ambulance (A0425, A0426, A0427, A0428, A0429, A0432, A0433, A0434, A0999)

new_df_3 = new_df[new_df['hcpcs'].isin(['A0425', 'A0426', 'A0427', 'A0428', 
                                        'A0429', 'A0432', 'A0433', 'A0434', 'A0999'])][['npi','year','lastname','state_id','state_fips','zip','tot_benes','tot_srvcs','tot_bene_day_srvcs','tot_srvcs_chrg']].drop_duplicates().reset_index(drop=True)
new_df_3_gb = new_df_3.groupby(['npi','lastname','state_id','state_fips','zip','year']).sum().reset_index(level=[0,1,2,3,4,5])
new_df_3_gb_gb = new_df_3_gb.groupby(['zip','year','state_id','state_fips']).sum()[['tot_benes','tot_srvcs','tot_bene_day_srvcs','tot_srvcs_chrg']].reset_index(level=[0,1,2,3])
new_df_3 = new_df_3_gb_gb.copy()

In [22]:
new_df_3.groupby(['zip','year','state_id','state_fips']).size().sort_values()

zip    year  state_id  state_fips
1001   2013  MA        25            1
60010  2016  IL        17            1
       2017  IL        17            1
       2018  IL        17            1
       2019  IL        17            1
                                    ..
30276  2018  GA        13            1
30277  2013  GA        13            1
30281  2013  GA        13            1
30268  2015  GA        13            1
99929  2019  AK        2             1
Length: 55335, dtype: int64

In [23]:
new_df_3.to_csv(r'all_amb.csv', index=False)

##### For Telehealth

In [24]:
filenames=['/Users/krhee/Dropbox (ASU)/Coding/RA_Coding/providerservice/MUP_PHY_R19_P04_V10_D13_Prov_Svc.csv',
          '/Users/krhee/Dropbox (ASU)/Coding/RA_Coding/providerservice/MUP_PHY_R19_P04_V10_D14_Prov_Svc.csv',
          '/Users/krhee/Dropbox (ASU)/Coding/RA_Coding/providerservice/MUP_PHY_R19_P04_V10_D15_Prov_Svc.csv',
          '/Users/krhee/Dropbox (ASU)/Coding/RA_Coding/providerservice/MUP_PHY_R19_P04_V10_D16_Prov_Svc.csv',
          '/Users/krhee/Dropbox (ASU)/Coding/RA_Coding/providerservice/MUP_PHY_R19_P04_V10_D17_Prov_Svc.csv',
           '/Users/krhee/Dropbox (ASU)/Coding/RA_Coding/providerservice/MUP_PHY_R20_P04_V10_D18_Prov_Svc.csv',
           '/Users/krhee/Dropbox (ASU)/Coding/RA_Coding/providerservice/MUP_PHY_R21_P04_V10_D19_Prov_Svc.csv']

In [28]:
new_df = pd.DataFrame()
for i in range(len(filenames)):
    temp_df = pd.read_csv(filenames[i], encoding='latin1')
    temp_df = temp_df[temp_df['HCPCS_Cd'].isin(['Q3014','G0406','G0407','G0408','G0425','G0426',
                                                'G0427','G0459','G0508','G0509'])].reset_index(drop=True)
    temp_df['year']= int(str('20')+filenames[i].split("_D")[-1][:2])
    new_df = new_df.append(temp_df).reset_index(drop=True)

In [29]:
new_df = new_df[['Rndrng_NPI','year','Rndrng_Prvdr_Last_Org_Name','HCPCS_Cd',
                         'Rndrng_Prvdr_State_Abrvtn','Rndrng_Prvdr_State_FIPS', 'Rndrng_Prvdr_Zip5',
                         'Tot_Benes','Tot_Srvcs','Tot_Bene_Day_Srvcs','Avg_Sbmtd_Chrg']]
new_df['Tot_Srvcs_Chrg'] = new_df['Tot_Srvcs']*new_df['Avg_Sbmtd_Chrg']
new_df = new_df.drop(columns=['Avg_Sbmtd_Chrg'])
new_df.columns=['npi','year','lastname','hcpcs','state_id','state_fips','zip','tot_benes','tot_srvcs','tot_bene_day_srvcs','tot_srvcs_chrg']

In [30]:
new_df_copy = new_df.copy()

In [31]:
# First, check if zip is missing --> All XXs
# new_df[new_df['zip'].isna()]
new_df = new_df[~new_df['zip'].isna()].reset_index(drop=True)
# Then let's match zip code to state
new_df['zip_3'] = [str(i)[:3] for i in new_df['zip']]
# Drop weird
new_df = new_df[~new_df['zip_3'].isin(['0','0.0', 'EH1','LA1','LA2','N9G'])].reset_index(drop=True)
new_df = new_df[~new_df['state_fips'].isna()].reset_index(drop=True)

In [32]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8995 entries, 0 to 8994
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   npi                 8995 non-null   int64  
 1   year                8995 non-null   int64  
 2   lastname            8995 non-null   object 
 3   hcpcs               8995 non-null   object 
 4   state_id            8995 non-null   object 
 5   state_fips          8995 non-null   float64
 6   zip                 8995 non-null   object 
 7   tot_benes           8995 non-null   int64  
 8   tot_srvcs           8995 non-null   float64
 9   tot_bene_day_srvcs  8995 non-null   int64  
 10  tot_srvcs_chrg      8995 non-null   float64
 11  zip_3               8995 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 843.4+ KB


In [34]:
# Check
zip_df = new_df[['zip','state_id','state_fips']].drop_duplicates()
zip_df.groupby(['zip']).nunique().sort_values(by=['state_id'])
zip_df.groupby(['zip']).nunique().sort_values(by=['state_fips'])
# State FIPS is more accurate --> Drop State ID
new_df = new_df.drop(columns=['state_id'])

In [35]:
# New state ID
state_dict = {1:'AL',2:'AK',4:'AZ',5:'AR',6:'CA',8:'CO',9:'CT',10:'DE',11:'DC',
              12:'FL',13:'GA',15:'HI',16:'ID',17:'IL',18:'IN',19:'IA',20:'KS',
              21:'KY',22:'LA',23:'ME',24:'MD',25:'MA',26:'MI',27:'MN',28:'MS',
              29:'MO',30:'MT',31:'NE',32:'NV',33:'NH',34:'NJ',35:'NM',36:'NY',
              37:'NC',38:'ND',39:'OH',40:'OK',41:'OR',42:'PA',44:'RI',45:'SC',
              46:'SD',47:'TN',48:'TX',49:'UT',50:'VT',51:'VA',53:'WA',54:'WV',
              55:'WI',56:'WY'}
state_df = pd.DataFrame.from_dict(data=state_dict, orient='index').reset_index(level=[0])
state_df.columns=['state_fips','state_id']
new_df = pd.merge(new_df, state_df, on=['state_fips'])

In [36]:
len(new_df)

8992

In [37]:
# Data types (important)
new_df['npi'] = new_df['npi'].astype(int)
new_df['year'] = new_df['year'].astype(int)
new_df['zip'] = new_df['zip'].astype(float)
new_df['zip'] = new_df['zip'].astype(int)
new_df['state_fips'] = new_df['state_fips'].astype(int)

In [39]:
# 1. Originating site (Q3014)

new_df_1 = new_df[new_df['hcpcs'].isin(['Q3014'])][['npi','year','lastname','state_id','state_fips','zip','tot_benes','tot_srvcs','tot_bene_day_srvcs','tot_srvcs_chrg']].drop_duplicates().reset_index(drop=True)
new_df_1_gb = new_df_1.groupby(['npi','lastname','state_id','state_fips','zip','year']).sum().reset_index(level=[0,1,2,3,4,5])
new_df_1_gb_gb = new_df_1_gb.groupby(['zip','year','state_id','state_fips']).sum().reset_index(level=[0,1,2,3])[['zip','year','state_id','state_fips',
                                                                  'tot_benes','tot_srvcs','tot_bene_day_srvcs','tot_srvcs_chrg']]
new_df_1 = new_df_1_gb_gb.copy()

In [40]:
new_df_1.groupby(['zip','year','state_id','state_fips']).size().sort_values()

zip    year  state_id  state_fips
1089   2019  MA        25            1
65712  2015  MO        29            1
       2016  MO        29            1
       2017  MO        29            1
65803  2014  MO        29            1
                                    ..
39307  2017  MS        28            1
       2018  MS        28            1
       2019  MS        28            1
39402  2019  MS        28            1
99901  2019  AK        2             1
Length: 3680, dtype: int64

In [41]:
# 2. All Telehealth

new_df_2 = new_df[['npi','year','lastname','state_id','state_fips','zip','tot_benes','tot_srvcs','tot_bene_day_srvcs','tot_srvcs_chrg']].drop_duplicates().reset_index(drop=True)
new_df_2_gb = new_df_2.groupby(['npi','lastname','state_id','state_fips','zip','year']).sum().reset_index(level=[0,1,2,3,4,5])
new_df_2_gb_gb = new_df_2_gb.groupby(['zip','year','state_id','state_fips']).sum().reset_index(level=[0,1,2,3])[['zip','year','state_id','state_fips',
                                                                  'tot_benes','tot_srvcs','tot_bene_day_srvcs','tot_srvcs_chrg']]
new_df_2 = new_df_2_gb_gb.copy()

In [42]:
new_df_2.groupby(['zip','year','state_id','state_fips']).size().sort_values()

zip    year  state_id  state_fips
1089   2019  MA        25            1
65803  2016  MO        29            1
       2014  MO        29            1
65712  2017  MO        29            1
       2016  MO        29            1
                                    ..
36535  2018  AL        1             1
36527  2019  AL        1             1
36526  2018  AL        1             1
36560  2017  AL        1             1
99901  2019  AK        2             1
Length: 4977, dtype: int64

In [43]:
new_df_1.to_csv(r'telehealth_numservices_q3014.csv', index=False)
new_df_2.to_csv(r'telehealth_numservices_all.csv', index=False)

### Number of physicians

In [44]:
# 1. Originating site (Q3014)

new_df_1 = new_df[new_df['hcpcs'].isin(['Q3014'])][['npi','year','lastname','state_id','state_fips','zip','tot_benes','tot_srvcs','tot_bene_day_srvcs','tot_srvcs_chrg']].drop_duplicates().reset_index(drop=True)
new_df_1_gb = new_df_1.groupby(['npi','lastname','state_id','state_fips','zip','year']).sum().reset_index(level=[0,1,2,3,4,5])
new_df_1_gb_gb = new_df_1_gb.groupby(['zip','year','state_id','state_fips']).nunique().reset_index(level=[0,1,2,3])[['zip','year','state_id','state_fips','npi']]
new_df_1 = new_df_1_gb_gb.copy()

In [45]:
new_df_1_gb_gb.groupby(['zip','year','state_id','state_fips']).size().sort_values()

zip    year  state_id  state_fips
1089   2019  MA        25            1
65712  2015  MO        29            1
       2016  MO        29            1
       2017  MO        29            1
65803  2014  MO        29            1
                                    ..
39307  2017  MS        28            1
       2018  MS        28            1
       2019  MS        28            1
39402  2019  MS        28            1
99901  2019  AK        2             1
Length: 3680, dtype: int64

In [46]:
new_df_1_gb_gb.to_csv(r'telehealth_numphysicians_q3014.csv', index=False)

In [47]:
# 2. All telehealth 

new_df_2 = new_df[['npi','year','lastname','state_id','state_fips','zip','tot_benes','tot_srvcs','tot_bene_day_srvcs','tot_srvcs_chrg']].drop_duplicates().reset_index(drop=True)
new_df_2_gb = new_df_2.groupby(['npi','lastname','state_id','state_fips','zip','year']).sum().reset_index(level=[0,1,2,3,4,5])
new_df_2_gb_gb = new_df_2_gb.groupby(['zip','year','state_id','state_fips']).nunique().reset_index(level=[0,1,2,3])[['zip','year','state_id','state_fips','npi']]
new_df_2 = new_df_2_gb_gb.copy()

In [48]:
new_df_2_gb_gb.groupby(['zip','year','state_id','state_fips']).size().sort_values()

zip    year  state_id  state_fips
1089   2019  MA        25            1
65803  2016  MO        29            1
       2014  MO        29            1
65712  2017  MO        29            1
       2016  MO        29            1
                                    ..
36535  2018  AL        1             1
36527  2019  AL        1             1
36526  2018  AL        1             1
36560  2017  AL        1             1
99901  2019  AK        2             1
Length: 4977, dtype: int64

In [49]:
new_df_2_gb_gb.to_csv(r'telehealth_numphysicians_all.csv', index=False)

----

### Merge

In [50]:
telephy_df1 = pd.read_csv(r'telehealth_numphysicians_q3014.csv')
telephy_df2 = pd.read_csv(r'telehealth_numphysicians_all.csv')
teleserv_df1 = pd.read_csv(r'telehealth_numservices_q3014.csv')
teleserv_df2 = pd.read_csv(r'telehealth_numservices_all.csv')

In [51]:
telephy_df1.columns=['zip','year','state_id','state_fips','numphysicians_tele_q3014']
telephy_df2.columns=['zip','year','state_id','state_fips','numphysicians_tele_all']
teleserv_df1.columns=['zip','year','state_id','state_fips','tot_benes_q3014',
                 'tot_srvcs_q3014','tot_bene_day_srvcs_q3014','tot_srvcs_chrg_q3014']
teleserv_df2.columns=['zip','year','state_id','state_fips','tot_benes_alltele',
                 'tot_srvcs_alltele','tot_bene_day_srvcs_alltele','tot_srvcs_chrg_alltele']

In [52]:
len(telephy_df1), len(telephy_df2),len(teleserv_df1),len(teleserv_df2)

(3680, 4977, 3680, 4977)

In [53]:
from functools import reduce

tele_df = reduce(lambda x,y: pd.merge(x,y, on=['zip','year','state_id','state_fips'],
                                      how='outer'), [telephy_df1,telephy_df2,teleserv_df1,teleserv_df2]).fillna(0)

In [56]:
tele_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4977 entries, 0 to 4976
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   zip                         4977 non-null   int64  
 1   year                        4977 non-null   int64  
 2   state_id                    4977 non-null   object 
 3   state_fips                  4977 non-null   int64  
 4   numphysicians_tele_q3014    4977 non-null   float64
 5   numphysicians_tele_all      4977 non-null   int64  
 6   tot_benes_q3014             4977 non-null   float64
 7   tot_srvcs_q3014             4977 non-null   float64
 8   tot_bene_day_srvcs_q3014    4977 non-null   float64
 9   tot_srvcs_chrg_q3014        4977 non-null   float64
 10  tot_benes_alltele           4977 non-null   int64  
 11  tot_srvcs_alltele           4977 non-null   float64
 12  tot_bene_day_srvcs_alltele  4977 non-null   int64  
 13  tot_srvcs_chrg_alltele      4977 

In [55]:
tele_df.to_csv(r'telehealth_numphysicians_numservices.csv', index=False)

----

#### Merge Tele & Ambulance

In [57]:
new_df_1 = pd.read_csv('nonemerg_amb.csv')
new_df_2 = pd.read_csv('emerg_amb.csv')
new_df_3 = pd.read_csv('all_amb.csv')

In [58]:
new_df_1.columns=['zip','year','state_id','state_fips','tot_benes_nonemerg',
                 'tot_srvcs_nonemerg','tot_bene_day_srvcs_nonemerg','tot_srvcs_chrg_nonemerg']
new_df_2.columns=['zip','year','state_id','state_fips','tot_benes_emerg',
                 'tot_srvcs_emerg','tot_bene_day_srvcs_emerg','tot_srvcs_chrg_emerg']
new_df_3.columns=['zip','year','state_id','state_fips','tot_benes_allamb',
                 'tot_srvcs_allamb','tot_bene_day_srvcs_allamb','tot_srvcs_chrg_allamb']

In [59]:
len(new_df_1), len(new_df_2), len(new_df_3)

(19770, 53501, 55335)

In [60]:
temp_df = pd.merge(new_df_1, new_df_2, on=['zip','year','state_id','state_fips'], how='outer')
temp_df = pd.merge(temp_df, new_df_3, on=['zip','year','state_id','state_fips'], how='outer')
temp_df = temp_df.fillna(0)
temp_df.info()
amb_df = temp_df.copy()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55335 entries, 0 to 55334
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   zip                          55335 non-null  int64  
 1   year                         55335 non-null  int64  
 2   state_id                     55335 non-null  object 
 3   state_fips                   55335 non-null  int64  
 4   tot_benes_nonemerg           55335 non-null  float64
 5   tot_srvcs_nonemerg           55335 non-null  float64
 6   tot_bene_day_srvcs_nonemerg  55335 non-null  float64
 7   tot_srvcs_chrg_nonemerg      55335 non-null  float64
 8   tot_benes_emerg              55335 non-null  float64
 9   tot_srvcs_emerg              55335 non-null  float64
 10  tot_bene_day_srvcs_emerg     55335 non-null  float64
 11  tot_srvcs_chrg_emerg         55335 non-null  float64
 12  tot_benes_allamb             55335 non-null  int64  
 13  tot_srvcs_allamb

In [61]:
temp_df = pd.merge(amb_df, tele_df, on=['zip','year','state_id','state_fips'], how='outer')
temp_df = temp_df.fillna(0)
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58177 entries, 0 to 58176
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   zip                          58177 non-null  int64  
 1   year                         58177 non-null  int64  
 2   state_id                     58177 non-null  object 
 3   state_fips                   58177 non-null  int64  
 4   tot_benes_nonemerg           58177 non-null  float64
 5   tot_srvcs_nonemerg           58177 non-null  float64
 6   tot_bene_day_srvcs_nonemerg  58177 non-null  float64
 7   tot_srvcs_chrg_nonemerg      58177 non-null  float64
 8   tot_benes_emerg              58177 non-null  float64
 9   tot_srvcs_emerg              58177 non-null  float64
 10  tot_bene_day_srvcs_emerg     58177 non-null  float64
 11  tot_srvcs_chrg_emerg         58177 non-null  float64
 12  tot_benes_allamb             58177 non-null  float64
 13  tot_srvcs_allamb

In [62]:
temp_df.to_csv(r'amb_tele_provider_service.csv', index=False)

##### From zip to county

In [245]:
amb_tele_df = pd.read_csv(r'amb_tele_provider_service.csv')

In [246]:
# group doctors by county
zip_county_df = pd.read_csv(r'/Users/krhee/Dropbox (ASU)/Coding/RA_Coding/providerservice/ziptocounty/ZIP_COUNTY.csv')
zip_county_df.columns=['zip','county','county_tot_ratio','year']

In [247]:
# keep only the counties that have max county_tot_ratio
zip_df_gb = zip_county_df.groupby(['zip','year']).max().reset_index(level=[0,1])[['zip','year','county_tot_ratio']].drop_duplicates()
zip_county_df = pd.merge(zip_county_df, zip_df_gb, on=['zip','year','county_tot_ratio'])
# If it's 50:50, keep the most frequent value by zip code
zip_county_df = zip_county_df.groupby(['zip']).agg(pd.Series.mode).reset_index(level=[0])
zip_county_df = zip_county_df[['zip','county']]

In [248]:
len(amb_tele_df), len(zip_county_df)

(57221, 39627)

In [249]:
temp_df = pd.merge(amb_tele_df, zip_county_df, on=['zip'], how='outer')
temp_df = temp_df[~temp_df['year'].isna()].reset_index(drop=True)
temp_df = temp_df[~temp_df['county'].isna()].reset_index(drop=True) # check
temp_df.info(null_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57175 entries, 0 to 57174
Data columns (total 27 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   zip                          57175 non-null  int64  
 1   year                         57175 non-null  float64
 2   state_id                     57175 non-null  object 
 3   state_fips                   57175 non-null  float64
 4   tot_benes_nonemerg           57175 non-null  float64
 5   tot_srvcs_nonemerg           57175 non-null  float64
 6   tot_bene_day_srvcs_nonemerg  57175 non-null  float64
 7   tot_srvcs_chrg_nonemerg      57175 non-null  float64
 8   tot_benes_emerg              57175 non-null  float64
 9   tot_srvcs_emerg              57175 non-null  float64
 10  tot_bene_day_srvcs_emerg     57175 non-null  float64
 11  tot_srvcs_chrg_emerg         57175 non-null  float64
 12  tot_benes_allamb             57175 non-null  float64
 13  tot_srvcs_allamb

/var/folders/ff/7xvchf4558v3w_jjs11bdwlw0000gp/T/ipykernel_43807/3812199944.py:4: FutureWarning: null_counts is deprecated. Use show_counts instead
  temp_df.info(null_counts=True)


In [250]:
temp_df_gb = temp_df.drop(columns=['zip']).groupby(['county','state_id','state_fips','year']).sum().reset_index(level=[0,1,2,3])

In [251]:
temp_df_gb.groupby(['county','year']).size().sort_values()

county  year  
1001    2013.0    1
40055   2019.0    1
40057   2013.0    1
        2014.0    1
        2015.0    1
                 ..
21115   2019.0    1
        2018.0    1
        2017.0    1
21117   2016.0    1
78030   2018.0    1
Length: 18188, dtype: int64

In [252]:
temp_df_gb.to_csv(r'amb_tele_county.csv', index=False)

##### From county to hsa

In [253]:
amb_tele_county_df = pd.read_csv(r'amb_tele_county.csv')

In [254]:
# group doctors by county
hsa_df = pd.read_csv(r'/Users/krhee/Dropbox (ASU)/Coding/RA_Coding/providerservice/hsatofips/fipstohsa.csv')
hsa_df.columns=['hsa','hsa-name','state-county','county']

In [255]:
hsa_df

,hsa,hsa-name,state-county,county
0,259,"Butler, AL",AL: Butler County (01013),1013
1,177,"Calhoun (Anniston), AL - Cleburne, AL",AL: Calhoun County (01015),1015
2,177,"Calhoun (Anniston), AL - Cleburne, AL",AL: Cleburne County (01029),1029
3,172,"Chambers, AL - Randolph, AL",AL: Chambers County (01017),1017
4,172,"Chambers, AL - Randolph, AL",AL: Randolph County (01111),1111
...,...,...,...,...
3212,775,"Teton, WY - Lincoln, WY",WY: Sublette County (56035),56035
3213,775,"Teton, WY - Lincoln, WY",WY: Teton County (56039),56039
3214,792,"Uinta, WY",WY: Uinta County (56041),56041
3215,804,"Weston, WY",WY: Weston County (56045),56045


In [258]:
len(amb_tele_county_df), len(hsa_df)

(18188, 3217)

In [259]:
temp_df = pd.merge(amb_tele_county_df, hsa_df, on=['county'], how='outer')
temp_df = temp_df[~temp_df['year'].isna()].reset_index(drop=True)
temp_df = temp_df[~temp_df['hsa'].isna()].reset_index(drop=True) # check (From virgin islands)
temp_df.info(null_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18178 entries, 0 to 18177
Data columns (total 29 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   county                       18178 non-null  int64  
 1   state_id                     18178 non-null  object 
 2   state_fips                   18178 non-null  float64
 3   year                         18178 non-null  float64
 4   tot_benes_nonemerg           18178 non-null  float64
 5   tot_srvcs_nonemerg           18178 non-null  float64
 6   tot_bene_day_srvcs_nonemerg  18178 non-null  float64
 7   tot_srvcs_chrg_nonemerg      18178 non-null  float64
 8   tot_benes_emerg              18178 non-null  float64
 9   tot_srvcs_emerg              18178 non-null  float64
 10  tot_bene_day_srvcs_emerg     18178 non-null  float64
 11  tot_srvcs_chrg_emerg         18178 non-null  float64
 12  tot_benes_allamb             18178 non-null  float64
 13  tot_srvcs_allamb

/var/folders/ff/7xvchf4558v3w_jjs11bdwlw0000gp/T/ipykernel_43807/1417906144.py:4: FutureWarning: null_counts is deprecated. Use show_counts instead
  temp_df.info(null_counts=True)


In [260]:
temp_df_gb = temp_df.groupby(['hsa','state_id','state_fips','year']).sum().reset_index(level=[0,1,2,3])
temp_df_gb.to_csv(r'amb_tele_hsa.csv', index=False)

In [265]:
temp_df_gb

,hsa,state_id,state_fips,year,county,tot_benes_nonemerg,tot_srvcs_nonemerg,tot_bene_day_srvcs_nonemerg,tot_srvcs_chrg_nonemerg,tot_benes_emerg,...,numphysicians_tele_q3014,numphysicians_tele_all,tot_benes_q3014,tot_srvcs_q3014,tot_bene_day_srvcs_q3014,tot_srvcs_chrg_q3014,tot_benes_alltele,tot_srvcs_alltele,tot_bene_day_srvcs_alltele,tot_srvcs_chrg_alltele
0,2.0,KY,21.0,2013.0,126518,45.0,1611.0,844.0,5.831820e+05,7369.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
1,2.0,KY,21.0,2014.0,126518,37.0,1133.0,613.0,4.101877e+05,7448.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
2,2.0,KY,21.0,2015.0,126518,50.0,974.0,529.0,3.531301e+05,7501.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
3,2.0,KY,21.0,2016.0,126518,2940.0,6543.1,5556.0,5.327887e+06,7671.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
4,2.0,KY,21.0,2017.0,126518,2951.0,6047.0,5153.0,5.388326e+06,8041.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6230,985.0,CO,8.0,2017.0,32294,126.0,151.0,146.0,7.057600e+04,907.0,...,1.0,1.0,68.0,104.0,104.0,3768.80,68.0,104.0,104.0,3768.80
6231,985.0,CO,8.0,2018.0,24183,82.0,90.0,89.0,5.861800e+04,1005.0,...,1.0,2.0,20.0,21.0,21.0,714.00,33.0,34.0,34.0,4718.00
6232,985.0,CO,8.0,2019.0,24183,71.0,73.0,73.0,6.206000e+04,834.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00
6233,986.0,NM,35.0,2017.0,35045,0.0,0.0,0.0,0.000000e+00,0.0,...,1.0,1.0,20.0,39.0,39.0,2505.76,20.0,39.0,39.0,2505.76


----